In [58]:
from datasets import load_dataset

train = load_dataset("CLUTRR/v1", name= "gen_train234_test2to10", split="train[:1600]")
test = load_dataset("CLUTRR/v1", name= "gen_train234_test2to10", split="test[:400]")

Found cached dataset v1 (C:/Users/Abhinav Gupta/.cache/huggingface/datasets/CLUTRR___v1/gen_train234_test2to10/1.0.0/5d12dad8acafc131b739b03fe83ab4164879e206337a0814a8a56e3fa86c80f2)
Found cached dataset v1 (C:/Users/Abhinav Gupta/.cache/huggingface/datasets/CLUTRR___v1/gen_train234_test2to10/1.0.0/5d12dad8acafc131b739b03fe83ab4164879e206337a0814a8a56e3fa86c80f2)


In [68]:
import pandas as pd
import numpy as np
import re
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, AutoModelForSequenceClassification
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import os
import accelerate
import transformers

In [69]:
train_dataset = pd.DataFrame(train)
test_dataset = pd.DataFrame(test)

train_dataset["input_text"] = train_dataset["clean_story"] + train_dataset['query'] + "," + train_dataset['genders']
test_dataset["input_text"] = test_dataset["clean_story"] + test_dataset['query'] + "," + test_dataset['genders']

def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove square brackets, parentheses, and single quotes using regex
    text = re.sub(r'[\[\]():]', ' ', text)
    # Remove single quotes
    text = text.replace("'", "")
    return text

train_dataset.input_text = train_dataset.input_text.apply(lambda text: preprocess_text(text))
test_dataset.input_text = test_dataset.input_text.apply(lambda text: preprocess_text(text))

train_dataset = train_dataset[['input_text', 'target']].rename(columns={'target': 'label'})
test_dataset = test_dataset[['input_text', 'target']].rename(columns={'target': 'label'})

train_dataset = Dataset.from_pandas(train_dataset, preserve_index=False)
test_dataset = Dataset.from_pandas(test_dataset, preserve_index=False)

In [70]:
tokenizer =  AutoTokenizer.from_pretrained("roberta-base", local_files_only=False, add_prefix_space=True)
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=18)

# print(tokenizer(train_dataset['input_text'][0], truncation=True))

def tokenization(data):
    return tokenizer(data['input_text'], padding = True, truncation=True)

train_dataset = train_dataset.map(tokenization, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenization, batched=True, batch_size=len(test_dataset))
train_dataset = train_dataset.remove_columns('input_text')
test_dataset = test_dataset.remove_columns('input_text')

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [72]:
# define accuracy metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [73]:
# define the training arguments
training_args = TrainingArguments(
    output_dir = './outputs',
    num_train_epochs=5,
    gradient_accumulation_steps = 16,    
    per_device_eval_batch_size= 8,
    evaluation_strategy = "epoch",
    disable_tqdm = False,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps = 8,
    dataloader_num_workers = 8,
)

In [74]:
# instantiate the trainer class and check for available devices
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)
device = 'cpu'
# train the model
trainer.train()

c:\Softwares\Anaconda\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


  0%|          | 0/60 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

In [ ]:
train_dataset

Dataset({
    features: ['input_text', 'labels'],
    num_rows: 2000
})

In [65]:
train_dataset = pd.DataFrame(train)
test_dataset = pd.DataFrame(test)

In [66]:
x = list(train_dataset.target.value_counts().keys())
y = list(test_dataset.target.value_counts().keys())
z = set(x).intersection(set(y))
z

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}